<h1>langchain의 tutorial 문서</h1>

<h2>Installation </h2>
python = 3.9 <br/>
pip install langchain <br/>

<h2>Lang chain tutorial </h2> 
프롬프트 + 모델 + 출력파서를 서로 결합해서 출력 다른 기능이 필요하면 결합해서 사용 가능

In [2]:
# open_api_key 필요
import os
import getpass
from langchain_openai import ChatOpenAI
os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


In [16]:
# tutorial 
from langchain_core.prompts import ChatPromptTemplate
# prompt 정의 
prompt = ChatPromptTemplate.from_template( "<Question>: {input}")
prompt

ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='<Question>: {input}'))])

In [47]:
# LLM model 선언 
from langchain_openai import ChatOpenAI
model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0,
)

In [7]:
# 출력 parser 정의 
from langchain_core.output_parsers import StrOutputParser
output = StrOutputParser()

In [19]:
# 구성요소들을 단일 체인으로 결합 
chain = prompt | model | output

In [20]:
# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})

'지구의 자전 주기는 약 24시간입니다. 이는 지구가 자전하는 데 걸리는 시간으로, 하루 동안 지구가 한 번 자전하는 것을 의미합니다. 이러한 자전은 지구의 자전축 주위를 돌면서 발생하며, 지구의 일정한 속도로 회전하고 있는 것을 나타냅니다.'

<h2>Lang chain nlp - sql tutorial </h2> 

In [22]:
# db는 sqlite만 지원 
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///coffee.db")

In [32]:
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT name FROM sqlite_master WHERE type='table';")

sqlite
[]


"[('menu',)]"

In [29]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# chain 결합 
chain = create_sql_query_chain(llm, db)

In [30]:
response = chain.invoke({"question": "What are the names of the items on the menu?"})
print(response)

SELECT "item_name" 
FROM "menu_items"
LIMIT 5;


In [ ]:
db.run(response)

<h2>Lang chain prompt-engineering</h2> 

In [42]:
examples = [
    {
        "instruction": "행동을 정의해주세요",
        "input": "아메리카노 한잔 따뜻하게 주세요  ",
        "answer": """
{"menu": "아메리카노"
"amount" : 1
"temperature" : "hot"
"action" : "order"}
""",
    },
    
    {
      "instruction": "행동을 정의해주세요",
        "input": "라떼 두잔 차갑게 주세요  ",
        "answer": """
{"menu": "라떼"
"amount" : 2
"temperature" : "ice"
"action" : "order"
}""" ,
    },
{
      "instruction": "행동을 정의해주세요",
        "input": "아이스라떼 두잔 주세요  ",
        "answer": """{
"menu": "라떼"
"amount" : 2
"temperature" : "ice"
"action" : "order"
}
""" ,
    },
    
    {
      "instruction": "행동을 정의해주세요",
        "input": "아이스아메리카노 두잔 주세요  ",
        "answer": """
{"menu": "아메리카노"
"amount" : 2
"temperature" : "ice"
"action" : "order"}
""" ,
        
    },
    
    {
"instruction": "행동을 정의해주세요",
"input": "메뉴판 보여주세요.",
"answer": """
{"action": "menu"}
""",    
    },
    
     {
"instruction": "행동을 정의해주세요",
"input": "가게 메뉴 알려주세요 .",
"answer": """
{"action": "menu"}
""",    
    },

         {
"instruction": "행동을 정의해 주세요",
"input": "메뉴판 보여줘 .",
"answer": """
{"action": "menu"}
""",    
    },
    
]

In [35]:
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
        ("ai", "{answer}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [51]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        few_shot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)

In [52]:
chain = final_prompt | llm

In [53]:
response = chain.invoke(
    {
        "instruction": "행동을 정의해주세요",
        "input": "카페 메뉴 보여주세요",
    }
)

print(response.content)

{"action": "menu"}


In [56]:
response = chain.invoke(
    {
        "instruction": "행동을 정의해주세요",
        "input": "아메리카노 주세요 ",
    }
)

print(response.content)

{"menu": "아메리카노"
"amount" : 1
"temperature" : "hot"
"action" : "order"}
